In [5]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.4'
spark_version = 'spark-3.4.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [8]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


df.show(5)

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales")


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""

# Execute the query
average_price_df = spark.sql(query)
average_price_df.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query
average_price_3bed_3bath_df = spark.sql(query)
average_price_3bed_3bath_df.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 200
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query
average_price_specific_criteria_df = spark.sql(query)
average_price_specific_criteria_df.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|280447.23|
|      2011|281413.45|
|      2012|295858.68|
|      2013|295142.13|
|      2014|294195.13|
|      2015|291788.36|
|      2016| 287812.6|
|      2017|282026.59|
+----------+---------+



In [12]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# SQL query to find the average price of homes greater than or equal to $350,000
query = """
SELECT view,
       ROUND(AVG(price), 2) as Average_Price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view
"""

# Execute the query
view_avg_price_df = spark.sql(query)
view_avg_price_df.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|   2|    397389.25|
|   3|     398867.6|
|   4|    399631.89|
|   5|    401471.82|
|   6|    395655.38|
|   7|    403005.77|
|   8|    398592.71|
|   9|    401393.34|
|  10|    401868.43|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.8861815929412842 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
# Cache the 'home_sales' temporary view
spark.catalog.cacheTable("home_sales")

# Check if the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached: {is_cached}")

Is 'home_sales' cached: True


In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')
# Check if the 'home_sales' table is cached (Already done in 7)
is_table_cached = spark.catalog.isCached('home_sales')
print(f"Is 'home_sales' table cached: {is_table_cached}")

Is 'home_sales' table cached: True


In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Write the data, partitioned by 'date_built' in Parquet format,
df.write.partitionBy("date_built").parquet("/mnt/data/home_sales_parquet")

In [ ]:
# 11. Read the parquet formatted data.

# Read the Parquet formatted data
parquet_df = spark.read.parquet("/content/drive/My Drive/home_sales_parquet")


parquet_df.show(5)

In [ ]:
# 12. Create a temporary table for the parquet data.
# Create a temporary table from the Parquet DataFrame
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# SQL query
parquet_result = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as Average_Price
    FROM parquet_home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")
parquet_result.show()


print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

is_table_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' still cached: {is_table_cached}")